In [1]:
import uproot
import numpy as np
import numba
import awkward1 as ak
import os
import optparse

from data import getData
from plot_utils import plotHist, plotCollection, plotEfficiency, combinePDFs
from truth_utils import truth_link, dr_match

In [2]:
cms_events = getData()

reading met skim file


In [16]:
genparts = cms_events['genParticles']
n1_mask = cms_events['genParticles'].pdgId==1000022
ak.min(genparts[n1_mask][:,:3].pt,axis=1)

<Array [38.9, 132, 68.2, ... 186, 99.2, 141] type='24854 * ?float32'>

In [ ]:
truth_builder = truth_link(cms_events['genParticles'], ak.ArrayBuilder())
truth_extension = truth_builder.snapshot()
ele_mask = np.abs(cms_events['genParticles']['pdgId']) == 11
first_mask = truth_extension['isFirst']
last_mask = np.abs(cms_events['genParticles']['status']) == 1 # equivalent to last                                                                                                            
z_mask = truth_extension['motherPdgId'] == 23
n2_mask = truth_extension['motherPdgId'] == 1000023
gen_ele_mask = ele_mask & (z_mask | n2_mask) & last_mask
truth_electrons = cms_events['genParticles'][gen_ele_mask]

In [ ]:
reco_electrons = cms_events['softElectrons']
#reco_electrons = cms_events['electrons']
reco_cuts = {
    "all": reco_electrons.pt > -1,
#    "looseMVA": reco_electrons.mvaId > 0,
#    "tightMVA": reco_electrons.mvaId > 5,
    }


In [ ]:
cut_name="all"
reco_mask = reco_cuts[cut_name]
signal_electrons = reco_electrons[reco_mask]

match_builder = dr_match(truth_electrons, signal_electrons, ak.ArrayBuilder(), doReco=False)
match_extension = match_builder.snapshot()
matching_truth_mask = match_extension.truth_to_reco_index >= 0
# matching_reco_mask = match_extension.truth_to_reco_index[matching_truth_mask]                                                                                                      

match_builderR = dr_match(truth_electrons, signal_electrons, ak.ArrayBuilder(), doReco=True)
match_extensionR = match_builderR.snapshot()
matching_reco_mask = match_extensionR.reco_to_truth_index >= 0
nonmatching_reco_mask = match_extensionR.reco_to_truth_index < 0

matched_truth = truth_electrons[matching_truth_mask]
unmatched_truth = truth_electrons[~matching_truth_mask]
matched_reco = signal_electrons[matching_reco_mask]
unmatched_reco = signal_electrons[~matching_reco_mask]
# unmatched_reco = signal_electrons[nonmatching_reco_mask]      

In [ ]:
plotCollection([matched_reco,unmatched_reco], cut_name+"_matching_reco", leg=["matched","unmatched"],
                           outDir="output_plots/diagnostic/match_comparison/reco_"+cut_name, normAttrs=False)

In [ ]:
attr='pt'
objs = [matched_reco,unmatched_reco]
vals = [ak.to_list(ak.flatten(o[attr])) for o in objs]
#plotHist('test', var=attr, vals=ak.flatten(objs[attr]), outDir='.', norm=True)
x = plotHist('test_'+attr, var=attr, vals=vals)

In [ ]:
#help(matched_reco)

In [ ]:
#reco_electrons[reco_electrons.pt > -1][0].pt
#matching_reco_mask = match_extensionR.reco_to_truth_index >= 0
#ak.to_list(matching_reco_mask[0])
print(matched_reco.pt[0:5])
print(ak.num(unmatched_reco.pt)[0:5])

In [ ]:
nprint=10
print(ak.num(truth_electrons)[:nprint])
print(ak.num(reco_electrons)[:nprint])
print(ak.to_list(truth_electrons.eta)[:nprint])
print(ak.to_list(reco_electrons.eta)[:nprint])
print(ak.to_list(truth_electrons.phi)[:nprint])
print(ak.to_list(reco_electrons.phi)[:nprint])

In [ ]:
print(ak.num(matched_truth.pt)[:nprint])
print(ak.to_list(matched_truth.pt)[:nprint])

In [ ]:
print(ak.num(unmatched_truth.pt)[:nprint])
print(ak.to_list(unmatched_truth.pt)[:nprint])

In [ ]:
print(ak.num(matched_reco.pt)[:nprint])
print(ak.to_list(matched_reco.pt)[:nprint])

In [ ]:
print(ak.num(unmatched_reco.pt)[:nprint])
print(ak.to_list(unmatched_reco.pt)[:nprint])

In [ ]:
print((ak.num(matched_truth.pt)+ak.num(unmatched_truth.pt))[:nprint])
print(ak.num(truth_electrons)[:nprint])

In [ ]:
print((ak.num(matched_reco.pt)+ak.num(unmatched_reco.pt))[:nprint])
print(ak.num(reco_electrons)[:nprint])

Now check that the contents are OK

In [ ]:
print((matched_truth.pt)[:nprint])
print((unmatched_truth.pt)[:nprint])
print((truth_electrons.pt)[:nprint])

In [ ]:
x = plotHist('test', vals=[[1,1,2,3,4],[5,5,5,6,7,8]])